In [1]:
import os
import pandas as pd
import numpy as np

import emoji, re

# 이미지 로딩
import requests
from PIL import Image
from io import BytesIO

In [16]:
all_users_profile = pd.read_parquet("C:/Users/flexmatch/Desktop/ssom/code/6.multimodal-sns-categorizer/all_users_profile.parquet")
all_users_profile.drop_duplicates(subset=['acnt_id'])

,acnt_id,acnt_nm,web_addr,acnt_sub_nm,intro_txt,profile_photo_url_addr,acnt_conn_yn,category_nm,follower_cnt,follow_cnt,media_cnt
0,17841400002751973,licoco___jin,https://link.inpock.co.kr/licoco___jin,김효진 리코코진👃🏻,[사랑의곧츄핑]아들셋의 엄마\n◡̈⃝카카오톡채널 : 리코코진 검색/등록\n🔻🔻구매링...,https://scontent-nrt1-2.xx.fbcdn.net/v/t51.288...,N,뷰티,30405,1559,16100
1,17841400006499942,kim__us_,None,김 동우,인플루언서 @우스 / @유튜버\n신혼 부부의 이야기 💑 우수리미\n#패션 #뷰티 #...,https://scontent-nrt1-2.xx.fbcdn.net/v/t51.288...,N,뷰티,312497,2245,1744
2,17841400011995621,aaari_678,https://open.kakao.com/o/st1HGfjc,조민희,블로거 할로비까/말쑥언니 최적4+\n아리따움 순창점💄 ➡️ @aritaum_sunc...,https://scontent-nrt1-2.xx.fbcdn.net/v/t51.288...,N,홈/리빙,1409,397,1276
3,17841400016295781,jihyolimm,None,임지효,Jihyo Lim\nBusiness mail 💌: limjihyo1108@gmail...,https://scontent-nrt1-2.xx.fbcdn.net/v/t51.288...,N,뷰티,37084,370,468
4,17841400025364370,9_5jw,None,지원쓰 ( 3 10 1),⚡️,https://scontent-nrt1-2.xx.fbcdn.net/v/t51.288...,N,뷰티,1240,759,102
...,...,...,...,...,...,...,...,...,...,...,...
6271,17841466793726270,tiffany_kikiki,None,티파니 유머키,재밌으니 배꼽 잡으삼\n맞팔 환영⭐️,https://scontent-nrt1-2.xx.fbcdn.net/v/t51.288...,N,뷰티,13140,69,760
6272,17841467052877958,seovelymom._,http://link.inpock.co.kr/velymom_,블리맘(김상미)의 육아톡톡,아이의 마음을 공부하는 6년차 육아맘👩‍👧‍👦\n놀이·아동심리상담사1급 자격증 보유...,https://scontent-nrt1-1.xx.fbcdn.net/v/t51.757...,N,뷰티,36397,905,488
6274,17841467167592837,nalssin_cook,https://link.inpock.co.kr/nalssin_cook,날씬쿡 | 떡볶이 먹을려고 클린식하는 계정,"📍매일 먹고, 매일 쓰는건 제일 좋은걸로\n📍알뜰살뜰 똑똑하게 살아가는게 추구미\n...",https://scontent-nrt1-2.xx.fbcdn.net/v/t51.288...,N,뷰티,111005,323,154
6275,17841468157240281,onetwo_fill,https://linktr.ee/onetwo_filll,하나둘 채워 chaewon🫧,🍒Beauty Creator✨ Skin care + make up\n🍒YouTube...,https://scontent-nrt1-1.xx.fbcdn.net/v/t51.288...,N,뷰티,1094,957,279


In [18]:
all_users_profile[all_users_profile['acnt_nm']=='imyeonduu']

,acnt_id,acnt_nm,web_addr,acnt_sub_nm,intro_txt,profile_photo_url_addr,acnt_conn_yn,category_nm,follower_cnt,follow_cnt,media_cnt
0,17841400000114584,imyeonduu,https://linkon.id/imyeonduu,연뚜 / 파티쉬,None,https://scontent-nrt1-1.xx.fbcdn.net/v/t51.288...,N,일상,257181,376,1094
0,17841400000114584,imyeonduu,https://linkon.id/imyeonduu,연뚜 / 파티쉬,None,https://scontent-nrt1-1.xx.fbcdn.net/v/t51.288...,N,뷰티,256583,377,1107


##### preprocessing & data merge

In [79]:
all_users = pd.read_parquet("C:/Users/flexmatch/Desktop/ssom/code/6.multimodal-sns-categorizer/all_users.parquet")
img_file_path = pd.read_csv("C:/Users/flexmatch/Desktop/ssom/code/6.multimodal-sns-categorizer/media_thumbnail_results.csv")

In [80]:
print(all_users['acnt_id'].nunique())
print(img_file_path['acnt_id'].nunique())

37016
36388


In [81]:
img_file_path = img_file_path.drop_duplicates(subset=['acnt_id', 'media_id'])

In [82]:
img_file_path.drop(['category', 'username', 'media_type'], axis=1, inplace=True)

In [83]:
all_users.groupby(['media_type_nm', 'reels_feed_type_nm'])['reels_feed_type_nm'].count()

media_type_nm   reels_feed_type_nm
CAROUSEL_ALBUM  FEED                  688909
IMAGE           FEED                  243526
VIDEO           FEED                   10728
                REELS                 486207
Name: reels_feed_type_nm, dtype: int64

In [84]:
all_users.groupby(['acnt_id'])['media_id'].count()

acnt_id
17841400000175995    44
17841400000253513    31
17841400000424290    45
17841400000500841    48
17841400000504076    28
                     ..
17841475448369973    22
17841475457249312     3
17841475488377754     3
17841475508932326    22
17841475830910947    25
Name: media_id, Length: 37016, dtype: int64

In [85]:
img_file_path.groupby(['acnt_id'])['media_id'].count()

acnt_id
17841400000175995    10
17841400000253513    10
17841400000424290    10
17841400000500841    10
17841400000504076    10
                     ..
17841475457249312     9
17841475488377754     7
17841475508932326    10
17841475830910947    10
17841476527044295    10
Name: media_id, Length: 36388, dtype: int64

In [86]:
img_file_path[img_file_path['acnt_id']==17841475488377754]

,acnt_id,media_id,image_path
163002,17841475488377754,17856817659457915,media_image\17841475488377754\1785681765945791...
163003,17841475488377754,18062257295458953,media_image\17841475488377754\1806225729545895...
163004,17841475488377754,18050152562252194,media_image\17841475488377754\1805015256225219...
163005,17841475488377754,17872841592380012,media_image\17841475488377754\1787284159238001...
163006,17841475488377754,17897236098229716,media_image\17841475488377754\1789723609822971...
163007,17841475488377754,18067647608088088,media_image\17841475488377754\1806764760808808...
163008,17841475488377754,18059522843244219,media_image\17841475488377754\1805952284324421...


In [87]:
all_users_2 = all_users.drop(['web_addr', 'profile_photo_url_addr', 'media_url_addr', 'media_unq_url_addr', 'tmnl_url_addr', 'feed_share_yn', 'cmnt_actvtn_yn'], axis=1)

In [88]:
print(len(all_users_2))
print(len(all_users_2.drop_duplicates(subset=['acnt_id', 'media_id'])))

# all_users_2.drop_duplicates(subset=['acnt_id'])

1429370
1428993


In [89]:
all_users_2['acnt_id'] = all_users_2['acnt_id'].astype(str).str.strip()
img_file_path['acnt_id'] = img_file_path['acnt_id'].astype(str).str.strip()

all_users_2['media_id'] = all_users_2['media_id'].astype(str).str.strip()
img_file_path['media_id'] = img_file_path['media_id'].astype(str).str.strip()


In [91]:
concat_df = pd.merge(all_users_2, img_file_path, on=['acnt_id', 'media_id'])
# concat_df.drop(['acnt_id_y'], axis=1, inplace=True)
# concat_df.rename(columns={'acnt_id_x':'acnt_id'}, inplace=True)
concat_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128481 entries, 0 to 128480
Data columns (total 19 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   acnt_id             128481 non-null  object 
 1   acnt_nm             128481 non-null  object 
 2   acnt_sub_nm         128481 non-null  object 
 3   intro_txt           128465 non-null  object 
 4   category_nm         128476 non-null  object 
 5   follower_cnt        128481 non-null  int64  
 6   follow_cnt          128481 non-null  int64  
 7   media_cnt           128481 non-null  int64  
 8   member_uid          128481 non-null  int64  
 9   main_category       128481 non-null  object 
 10  top_3_category      128481 non-null  object 
 11  media_id            128481 non-null  object 
 12  media_type_nm       128481 non-null  object 
 13  reels_feed_type_nm  128481 non-null  object 
 14  media_cn            125376 non-null  object 
 15  acnt_conn_yn        128481 non-nul

In [ ]:
# 여기까지 확인 결과, image_thumbnail의 수집 시점이 다르기 때문에 기존에 가지고 있는 media_id보다 많을 수 있도, media_id가 같지 않은게 많을수도 있음.
# 또한, reels와 중복은 삭제
# 기존 all_users_media는 한 유저당 50개씩의 미디어 데이터를 수집했지만, 동일님 수집코드는 한 사람당 기본 게사물 개수인 10개여서 merge 이후에 데이터가 더 줄어듦
concat_df.groupby(['acnt_id'])['media_id'].count().sort_values(ascending=False).to_frame('count').reset_index()['count'].value_counts()


count
10    2744
7     2698
8     2596
6     2489
9     2307
5     2049
4     1845
3     1473
2     1251
1     1144
Name: count, dtype: int64

text preprocessing

In [64]:
from hanspell import spell_checker
import xml.etree.ElementTree as ET

spelled_checked_text = []
for lst in concat_df['media_cn']:
    try:
        result = spell_checker.check(lst)
        spelled_text = result.checked
    except ET.ParseError:
        spelled_text = lst  # 오류 발생 시 원래 텍스트 사용
    except Exception as e:
        spelled_text = lst  # 기타 예외 발생 시 원래 텍스트 사용
        print(f"Error processing text: {lst}\nException: {e}")
    
    spelled_checked_text.append(spelled_text)

concat_df['spelled_checked_media_cn'] = spelled_checked_text

Error processing text: None
Exception: object of type 'NoneType' has no len()
Error processing text: #광고

요즘 헬스장 탈의실에서 은근 자주 보이던 그 바디워시, 드디어 써봤어요!

아르투아 송 퍼퓸 바디워시✨

"스치면 쳐다보는 향수로션"이라는 별명이 괜히 붙은 게 아니더라고요.

부드러운 거품에 은은하게 올라오는 비누향+프리지아 향기가

진짜 산마노 프리지아 향수랑 99.9% 유사해서 놀랐어요.

거기에다가 호텔 느낌 나는 백화점 향기라서 기분까지 리프레시!

✔ 퍼퓸 바디워시답게 잔향 지속력도 길고 은은하게 퍼져요

✔ 손, 팔에 부드럽게 감기는 향이 너무 좋고

✔ 향수 원료 그대로 사용해서 퀄리티도 만족

✔ 피부 당김 없이 촉촉하게 마무리되는 것도 포인트!

✔ 패키지도 향기처럼 고급스럽고 깔끔해서 선물용으로도 굿

전 올리브영에서 구매했는데, 바디로션도 같이 쓰면 향기관리 끝입니다.

요즘 헬창들 사이에서도 인기 많은 이유가 있네요💪🏻

#artois #바디로션 #아르투아 #퍼퓸바디로션추천 #향좋은바디로션 #퍼퓸바디워시 #향좋은바디워시 #비누향바디워시
Exception: Expecting value: line 1 column 1 (char 0)
Error processing text: None
Exception: object of type 'NoneType' has no len()
Error processing text: 🌱이제 다음달이면 본격적인 휴가철인데 
그동안 맛집 다니면서 비축해둔 뱃살 때문에
수영복은 입을 엄두도 안나구요 😂

🌱나이가 들면서 기초대사량이 떨어지니 
조금 굶으면 되겠지 하던 다이어트도
절식과 폭식의 반복으로 요요가 오니까 
🌱체질개선부터 해야겠다 싶더라구요

🌱무작적 굶는거 말고 적당히 식단과 운동 병행하면서 
요요없이 길게가는 다이어트를 하고싶어서 
🌱다이트한의원 강남점 방문해 보았어요😊

🌱개인 맞춤 한약 제조로 평소 당뇨, 갑상선, 호

In [68]:
concat_df['spelled_checked_media_cn'] = concat_df['spelled_checked_media_cn'].fillna(concat_df['media_cn'])

# 빈 문자열("")이나 None도 대체
concat_df['spelled_checked_media_cn'] = concat_df.apply(
    lambda row: row['media_cn'] if pd.isna(row['spelled_checked_media_cn']) or row['spelled_checked_media_cn'] == "" else row['spelled_checked_media_cn'],
    axis=1
)

In [72]:
def clean_text(text):
    if not isinstance(text, str):
        return ''
    
    text = emoji.replace_emoji(text, replace='')
    text = re.sub(r'[^가-힣a-zA-Z0-9\s]', '', text)
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text) # 특수 문자 제거
    text = re.sub(r'\s+', ' ', text) # 불필요한 공백 제거

    return text.strip()

In [73]:
concat_df['spelled_checked_media_cn'] = concat_df['spelled_checked_media_cn'].apply(clean_text)
concat_df['acnt_sub_nm'] = concat_df['acnt_sub_nm'].apply(clean_text)
concat_df['intro_txt'] = concat_df['intro_txt'].apply(clean_text)

In [74]:
concat_df.to_parquet("all_users_media_prepro.parquet")

2nd preprocessing

In [70]:
# img_file_path = pd.read_csv("C:/Users/flexmatch/Desktop/ssom/code/6.multimodal-sns-categorizer/media_thumbnail_results.csv")
# img_file_path

In [ ]:
media_data = pd.read_parquet("C:/Users/flexmatch/Desktop/ssom/code/6.multimodal-sns-categorizer/all_users_media_prepro.parquet")
media_data.info()

In [6]:
media_data = media_data.drop(['media_cn'], axis=1)

In [73]:
media_data.drop_duplicates(['acnt_id', 'media_id'])

,acnt_id,acnt_nm,acnt_sub_nm,intro_txt,category_nm,follower_cnt,follow_cnt,media_cnt,member_uid,main_category,top_3_category,media_id,media_type_nm,reels_feed_type_nm,acnt_conn_yn,cmnt_cnt,like_cnt,image_path,spelled_checked_media_cn
0,17841400038455668,kos9803,마곡원주민,여행 육아 가족 힐링 캠핑 it 마곡,뷰티,24052,866,870,0,여행/관광,여행/관광@유명장소/핫플@홈/리빙,18088296511639307,CAROUSEL_ALBUM,FEED,N,0,4.0,media_image\17841400038455668\1808829651163930...,협찬 다양한 기능을 한번에 활용할 수 있는 야채탈수기와 채칼 6종 동시 사용까지 가...
1,17841400038455668,kos9803,마곡원주민,여행 육아 가족 힐링 캠핑 it 마곡,뷰티,24052,866,870,0,여행/관광,여행/관광@유명장소/핫플@홈/리빙,17888713530296800,VIDEO,REELS,N,0,6.0,media_image\17841400038455668\1788871353029680...,광고국내 자체 제작 도안으로 diy 쉽게 가능한테슬라 모델 y 주니퍼 자동차 ppf...
2,17841400038455668,kos9803,마곡원주민,여행 육아 가족 힐링 캠핑 it 마곡,뷰티,24052,866,870,0,여행/관광,여행/관광@유명장소/핫플@홈/리빙,17971563212887318,CAROUSEL_ALBUM,FEED,N,0,4.0,media_image\17841400038455668\1797156321288731...,광고 이사 입주가구 신혼 혼수가구 및 가전제품까지 없는게 없는 김포 가구 가전 판매...
3,17841400038455668,kos9803,마곡원주민,여행 육아 가족 힐링 캠핑 it 마곡,뷰티,24052,866,870,0,여행/관광,여행/관광@유명장소/핫플@홈/리빙,18146171110377497,CAROUSEL_ALBUM,FEED,N,0,6.0,media_image\17841400038455668\1814617111037749...,친한 대표님이 초대를 해주셔서 김포 전시장을 방문했는데요 그동안 나이가 있어서 창업...
4,17841400038455668,kos9803,마곡원주민,여행 육아 가족 힐링 캠핑 it 마곡,뷰티,24052,866,870,0,여행/관광,여행/관광@유명장소/핫플@홈/리빙,17850738861474402,VIDEO,REELS,N,0,3.0,media_image\17841400038455668\1785073886147440...,안녕하세요 드디어 어두웠던 주방에 광명을 찾았네요이사 온 지 10년 차가 되니 기존...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132090,17841470093907189,rozy_yaksa,로지약사 프로 n 잡러 장지나 약사,4살 딸을 위해 영양제 만드는 9년차 엄마 약사 우리 아이 건강 상담 환영합니다 올...,뷰티,272,110,19,0,베이비/키즈,베이비/키즈@다이어트/건강보조식품,18058795693738192,VIDEO,REELS,N,4,36.0,media_image\17841470093907189\1805879569373819...,육퇴후 올리는 유산균의 좋은 균주 고르는 법 점점 클수록 잘 때는 엄마만 찾는 거실...
132091,17841470093907189,rozy_yaksa,로지약사 프로 n 잡러 장지나 약사,4살 딸을 위해 영양제 만드는 9년차 엄마 약사 우리 아이 건강 상담 환영합니다 올...,뷰티,272,110,19,0,베이비/키즈,베이비/키즈@다이어트/건강보조식품,17947612817864394,VIDEO,REELS,N,4,24.0,media_image\17841470093907189\1794761281786439...,약사가 알려주는 철분부족 7가지 대표증상 여성분들의 경우에는 매달 생리로 인하여 혈...
132092,17841470093907189,rozy_yaksa,로지약사 프로 n 잡러 장지나 약사,4살 딸을 위해 영양제 만드는 9년차 엄마 약사 우리 아이 건강 상담 환영합니다 올...,뷰티,272,110,19,0,베이비/키즈,베이비/키즈@다이어트/건강보조식품,18060847024829065,VIDEO,REELS,N,3,22.0,media_image\17841470093907189\1806084702482906...,첫째가 너무 예뻐 둘째가 너무 낳고 싶은데요 베투키도 키우고 있는지라둘째는 엄두도 ...
132093,17841470093907189,rozy_yaksa,로지약사 프로 n 잡러 장지나 약사,4살 딸을 위해 영양제 만드는 9년차 엄마 약사 우리 아이 건강 상담 환영합니다 올...,뷰티,272,110,19,0,베이비/키즈,베이비/키즈@다이어트/건강보조식품,17978237999635607,VIDEO,REELS,N,4,26.0,media_image\17841470093907189\1797823799963560...,엄마약사는 절대 안 쓰는 미세플라스틱 폭발 생활용품 4가지 평소에 사용하기 쉽고 우...


resnet

In [ ]:
# import torch
# import torch.nn as nn
# import torchvision.models as models
# import torchvision.transforms as transforms
# from torch.autograd import Variable

In [ ]:
# #Resize the image to 224x224 px
# # input size도 변경(resnet 모델 변경시)

# scaler = transforms.Resize((224, 224))
# # normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
# #                                  std=[0.229, 0.224, 0.225]) # scaling은 대부분 이 사이즈로 실행
# normalize = transforms.Normalize((0.5,),(0.5,))

# to_tensor = transforms.ToTensor()

# # Use the model object to select the desired layer
# layer = model._modules.get('avgpool')

# def extract_feature_vector(img):
#     # 2. Create a PyTorch Variable with the transformed image
#     #Unsqueeze actually converts to a tensor by adding the number of images as another dimension.
#     t_img = Variable(normalize(to_tensor(scaler(img))).unsqueeze(0))

#     # 3. Create a vector of zeros that will hold our feature vector
#     #    The 'avgpool' layer has an output size of 512 -> output size로 변경해야함
#     my_embedding = torch.zeros(1, 2048, 1, 1)

#     # 4. Define a function that will copy the output of a layer
#     def copy_data(m, i, o):
#         my_embedding.copy_(o.data)

#     # 5. Attach that function to our selected layer
#     h = layer.register_forward_hook(copy_data)

#     # 6. Run the model on our transformed image
#     model(t_img)

#     # 7. Detach our copy function from the layer
#     h.remove()

#     # 8. Return the feature vector
#     return my_embedding.squeeze().numpy()

In [ ]:
# image = Image.open("/content/drive/MyDrive/Colab Notebooks/sample/20070@N98/173632.jpg")

# a = extract_feature_vector(image)